# 🏠 PROJET IMMOBILIER - CLARA BENALI
## Pipeline Complet : DVF + Population INSEE (Paris 75 & Hauts-de-Seine 92)

---

### 📋 Objectif
Créer un dataset enrichi combinant :
- **DVF** : Transactions immobilières (2020-2025)
- **INSEE Population** : Données démographiques par commune

### 🎯 Structure
1. **Importation des bibliothèques**
2. **Chargement DVF** (déjà disponible localement)
3. **Téléchargement Population INSEE**
4. **Nettoyage & Préparation**
5. **Jointure DVF + Population**
6. **Export final**

---
## 📚 ÉTAPE 1 : IMPORTATION DES BIBLIOTHÈQUES

In [1]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
from pathlib import Path
from datetime import datetime

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("✅ Bibliothèques importées")
print(f"📅 {datetime.now().strftime('%d/%m/%Y %H:%M')}")

✅ Bibliothèques importées
📅 01/11/2025 16:10


---
## 📥 ÉTAPE 2 : CHARGEMENT DVF (Depuis fichiers locaux)

In [2]:
print("📂 Chargement des fichiers DVF locaux...\n")

# Liste des fichiers à charger
fichiers_dvf = []
annees = [2020, 2021, 2022, 2023, 2024, 2025]
departements = ['75', '92']

for annee in annees:
    for dept in departements:
        chemin = f'../data project/{annee}/{dept}.csv'
        try:
            df = pd.read_csv(chemin)
            fichiers_dvf.append(df)
            print(f"   ✅ {annee}/{dept}.csv : {len(df):,} lignes")
        except FileNotFoundError:
            print(f"   ⚠️  Fichier manquant : {chemin}")

# Concaténer
dvf_brut = pd.concat(fichiers_dvf, ignore_index=True)

print(f"\n✅ Total DVF brut : {len(dvf_brut):,} transactions")
print(f"   Colonnes : {len(dvf_brut.columns)}")
print(f"   Période : {dvf_brut.shape}")

📂 Chargement des fichiers DVF locaux...

   ✅ 2020/75.csv : 28,777 lignes


C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\1378779247.py:12: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin)


   ✅ 2020/92.csv : 47,597 lignes


C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\1378779247.py:12: DtypeWarning: Columns (18,20,22,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin)


   ✅ 2021/75.csv : 81,516 lignes
   ✅ 2021/92.csv : 80,721 lignes


...

   ✅ 2023/75.csv : 80,964 lignes


C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\1378779247.py:12: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin)


   ✅ 2023/92.csv : 61,616 lignes


C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\1378779247.py:12: DtypeWarning: Columns (8,18,20,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(chemin)


   ✅ 2024/75.csv : 74,677 lignes
   ✅ 2024/92.csv : 58,382 lignes
   ✅ 2025/75.csv : 31,405 lignes
   ✅ 2025/92.csv : 21,791 lignes

✅ Total DVF brut : 747,000 transactions
   Colonnes : 40
   Période : (747000, 40)


---
## 🧹 ÉTAPE 3 : NETTOYAGE DVF

In [3]:
print("🧹 Nettoyage des données DVF...\n")

# Sélectionner colonnes pertinentes
colonnes_pertinentes = [
    'id_mutation', 'date_mutation', 'nature_mutation',
    'code_departement', 'code_commune', 'nom_commune', 'code_postal',
    'latitude', 'longitude',
    'type_local', 'valeur_fonciere', 'surface_reelle_bati', 'nombre_pieces_principales'
]

colonnes_existantes = [c for c in colonnes_pertinentes if c in dvf_brut.columns]
dvf = dvf_brut[colonnes_existantes].copy()

print(f"1️⃣  Colonnes sélectionnées : {len(dvf.columns)}")

# Filtrer ventes uniquement
dvf = dvf[dvf['nature_mutation'] == 'Vente'].copy()
print(f"2️⃣  Ventes uniquement : {len(dvf):,}")

# Filtrer types de biens
types_valides = ['Maison', 'Appartement']
dvf = dvf[dvf['type_local'].isin(types_valides)].copy()
print(f"3️⃣  Types valides (Maison/Appartement) : {len(dvf):,}")

# Valeurs cohérentes
dvf = dvf[
    (dvf['valeur_fonciere'] >= 1000) &
    (dvf['valeur_fonciere'] <= 20_000_000) &
    (dvf['surface_reelle_bati'] >= 9) &
    (dvf['surface_reelle_bati'] <= 1000)
].copy()
print(f"4️⃣  Valeurs cohérentes : {len(dvf):,}")

# Calculer prix/m²
dvf['prix_m2'] = dvf['valeur_fonciere'] / dvf['surface_reelle_bati']

# Supprimer outliers prix/m²
q01 = dvf['prix_m2'].quantile(0.01)
q99 = dvf['prix_m2'].quantile(0.99)
dvf = dvf[(dvf['prix_m2'] >= q01) & (dvf['prix_m2'] <= q99)].copy()
print(f"5️⃣  Sans outliers prix/m² : {len(dvf):,}")

# Extraire année
dvf['date_mutation'] = pd.to_datetime(dvf['date_mutation'])
dvf['annee'] = dvf['date_mutation'].dt.year
dvf['trimestre'] = dvf['date_mutation'].dt.quarter

# Type de bien simplifié
dvf['type_bien'] = dvf['type_local']

# Supprimer lignes sans géolocalisation
dvf = dvf.dropna(subset=['latitude', 'longitude'])
print(f"6️⃣  Avec coordonnées GPS : {len(dvf):,}")

# ⭐ CONVERTIR CODE_COMMUNE EN STRING
dvf['code_commune'] = dvf['code_commune'].astype(str)

print(f"\n✅ DVF nettoyé : {len(dvf):,} transactions")
print(f"\n📊 Répartition par département :")
print(dvf['code_departement'].value_counts())

🧹 Nettoyage des données DVF...

1️⃣  Colonnes sélectionnées : 13
2️⃣  Ventes uniquement : 679,657
3️⃣  Types valides (Maison/Appartement) : 302,690
4️⃣  Valeurs cohérentes : 286,538
5️⃣  Sans outliers prix/m² : 280,838
6️⃣  Avec coordonnées GPS : 279,554

✅ DVF nettoyé : 279,554 transactions

📊 Répartition par département :
code_departement
75    169918
92    109636
Name: count, dtype: int64


---
## 📥 ÉTAPE 4 : TÉLÉCHARGEMENT POPULATION INSEE

In [4]:
print("📥 Téléchargement Population INSEE...\n")

url_pop = "https://www.insee.fr/fr/statistiques/fichier/6683035/ensemble.zip"

try:
    response = requests.get(url_pop, timeout=120)
    print(f"✅ Téléchargement : {len(response.content) / 1024 / 1024:.1f} Mo\n")
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        with z.open('donnees_communes.csv') as f:
            pop_brut = pd.read_csv(
                f,
                sep=';',
                dtype={'CODCOM': str, 'COM': str, 'CODDEP': str}
            )
    
    print(f"✅ {len(pop_brut):,} communes chargées")
    
except Exception as e:
    print(f"❌ Erreur : {e}")
    pop_brut = None

📥 Téléchargement Population INSEE...

✅ Téléchargement : 0.9 Mo

✅ 34,980 communes chargées


---
## 🔧 ÉTAPE 5 : PRÉPARATION POPULATION (75 & 92)

In [5]:
if pop_brut is not None:
    print("🔧 Préparation des données Population...\n")
    
    # ⭐ FILTRER 75 ET 92 **AVANT** DE CRÉER LE DATAFRAME
    pop_75_92 = pop_brut[pop_brut['CODDEP'].isin(['75', '92'])].copy()
    
    print(f"1️⃣  Filtrage 75 & 92 : {len(pop_75_92)} communes")
    print(f"    - Paris (75) : {len(pop_75_92[pop_75_92['CODDEP']=='75'])}")
    print(f"    - Hauts-de-Seine (92) : {len(pop_75_92[pop_75_92['CODDEP']=='92'])}")
    
    # Créer DataFrame final
    pop_final = pd.DataFrame({
        'code_commune': pop_75_92['CODDEP'] + pop_75_92['CODCOM'],  # ⭐ Concaténer DEPT + COM
        'nom_commune': pop_75_92['COM'],
        'population': pd.to_numeric(pop_75_92['PMUN'], errors='coerce'),
        'code_departement': pop_75_92['CODDEP']
    })
    
    # Nettoyer
    pop_final = pop_final.dropna(subset=['population'])
    pop_final['population'] = pop_final['population'].astype(int)
    
    print(f"\n2️⃣  DataFrame créé : {len(pop_final)} communes")
    print(f"    Total population : {pop_final['population'].sum():,} habitants")
    
    # Afficher exemples
    print(f"\n3️⃣  Exemples codes communes :")
    print(pop_final[['code_commune', 'nom_commune', 'population']].head(10))
    
else:
    print("❌ Population non disponible")
    pop_final = None

🔧 Préparation des données Population...

1️⃣  Filtrage 75 & 92 : 56 communes
    - Paris (75) : 20
    - Hauts-de-Seine (92) : 36

2️⃣  DataFrame créé : 56 communes
    Total population : 3,772,119 habitants

3️⃣  Exemples codes communes :
      code_commune               nom_commune  population
29283        75101  Paris 1er Arrondissement       16030
29284        75102   Paris 2e Arrondissement       21130
29285        75103   Paris 3e Arrondissement       33402
29286        75104   Paris 4e Arrondissement       29064
29287        75105   Paris 5e Arrondissement       57380
29288        75106   Paris 6e Arrondissement       39625
29289        75107   Paris 7e Arrondissement       48520
29290        75108   Paris 8e Arrondissement       35631
29291        75109   Paris 9e Arrondissement       60168
29292        75110  Paris 10e Arrondissement       83459


---
## 🔍 ÉTAPE 6 : VÉRIFICATION AVANT JOINTURE

In [6]:
if pop_final is not None:
    print("🔍 Vérification compatibilité codes communes...\n")
    
    # Codes DVF
    codes_dvf = set(dvf['code_commune'].unique())
    print(f"DVF : {len(codes_dvf)} codes uniques")
    print(f"  Exemples : {list(codes_dvf)[:10]}")
    
    # Codes Population
    codes_pop = set(pop_final['code_commune'].unique())
    print(f"\nPopulation : {len(codes_pop)} codes uniques")
    print(f"  Exemples : {list(codes_pop)[:10]}")
    
    # Intersection
    codes_communs = codes_dvf.intersection(codes_pop)
    print(f"\n✅ Codes en commun : {len(codes_communs)}")
    
    if len(codes_communs) == 0:
        print("\n❌ PROBLÈME : Aucun code en commun !")
        print("\n🔧 Diagnostic :")
        print(f"  Type DVF : {dvf['code_commune'].dtype}")
        print(f"  Type Pop : {pop_final['code_commune'].dtype}")
    else:
        print(f"\n🎉 Jointure possible !")
        taux_couverture = len(codes_communs) / len(codes_dvf) * 100
        print(f"  Couverture attendue : {taux_couverture:.1f}%")

🔍 Vérification compatibilité codes communes...

DVF : 56 codes uniques
  Exemples : ['75112', '92026', '92049', '75117', '92050', '92048', '92046', '92023', '92071', '75102']

Population : 56 codes uniques
  Exemples : ['75112', '92026', '92049', '75117', '92050', '92048', '92046', '92023', '92071', '75102']

✅ Codes en commun : 56

🎉 Jointure possible !
  Couverture attendue : 100.0%


---
## 🔗 ÉTAPE 7 : JOINTURE DVF + POPULATION

In [7]:
if pop_final is not None:
    print("🔗 Jointure DVF + Population...\n")
    
    # Jointure LEFT (garder toutes les transactions DVF)
    dvf_enrichi = dvf.merge(
        pop_final[['code_commune', 'population']],
        on='code_commune',
        how='left'
    )
    
    # Statistiques
    nb_avec_pop = dvf_enrichi['population'].notna().sum()
    taux = (nb_avec_pop / len(dvf_enrichi)) * 100
    
    print(f"✅ Jointure effectuée")
    print(f"\n📊 Résultat :")
    print(f"  Total transactions : {len(dvf_enrichi):,}")
    print(f"  Avec population : {nb_avec_pop:,} ({taux:.2f}%)")
    print(f"  Sans population : {len(dvf_enrichi) - nb_avec_pop:,}")
    
    # Par département
    print(f"\n📈 Par département :")
    stats_dept = dvf_enrichi.groupby('code_departement').agg({
        'id_mutation': 'count',
        'population': lambda x: x.notna().sum()
    })
    stats_dept.columns = ['Nb_transactions', 'Avec_population']
    stats_dept['Taux_%'] = (stats_dept['Avec_population'] / stats_dept['Nb_transactions'] * 100).round(1)
    print(stats_dept)
    
else:
    dvf_enrichi = dvf.copy()
    print("⚠️  Pas de jointure (population indisponible)")

🔗 Jointure DVF + Population...

✅ Jointure effectuée

📊 Résultat :
  Total transactions : 279,554
  Avec population : 279,554 (100.00%)
  Sans population : 0

📈 Par département :
                  Nb_transactions  Avec_population  Taux_%
code_departement                                          
75                         169918           169918  100.00
92                         109636           109636  100.00


---
## 👀 ÉTAPE 8 : APERÇU DATASET FINAL

In [8]:
print("👀 APERÇU DU DATASET FINAL\n")

# Colonnes à afficher
cols_affichage = [
    'code_commune', 'nom_commune', 'type_bien', 
    'valeur_fonciere', 'surface_reelle_bati', 'prix_m2', 
    'population', 'annee'
]

cols_disponibles = [c for c in cols_affichage if c in dvf_enrichi.columns]

print(dvf_enrichi[cols_disponibles].head(15))

print(f"\n📊 Statistiques descriptives :")
print(dvf_enrichi[['valeur_fonciere', 'surface_reelle_bati', 'prix_m2', 'population']].describe())

👀 APERÇU DU DATASET FINAL

   code_commune               nom_commune    type_bien  valeur_fonciere  \
0         75103   Paris 3e Arrondissement  Appartement        148000.00   
1         75101  Paris 1er Arrondissement  Appartement        270000.00   
2         75101  Paris 1er Arrondissement  Appartement       2603550.00   
3         75101  Paris 1er Arrondissement  Appartement       1112000.00   
4         75101  Paris 1er Arrondissement  Appartement       1670000.00   
5         75101  Paris 1er Arrondissement  Appartement        314150.00   
6         75108   Paris 8e Arrondissement  Appartement        891000.00   
7         75103   Paris 3e Arrondissement  Appartement       1993000.00   
8         75108   Paris 8e Arrondissement  Appartement         19440.00   
9         75108   Paris 8e Arrondissement  Appartement        640000.00   
10        75103   Paris 3e Arrondissement  Appartement       1778000.00   
11        75108   Paris 8e Arrondissement  Appartement        968000.00  

In [29]:
dvf.columns

Index(['id_mutation', 'date_mutation', 'nature_mutation', 'code_departement',
       'code_commune', 'nom_commune', 'code_postal', 'latitude', 'longitude',
       'type_local', 'valeur_fonciere', 'surface_reelle_bati',
       'nombre_pieces_principales', 'prix_m2', 'annee', 'trimestre',
       'type_bien'],
      dtype='object')

In [36]:
dvf_enrichi.to_csv("C:/Users/kombi/OneDrive/Bureau/p/data revenus/2023/dvf_enricheed.csv", index=False)

In [30]:
dvf_enrichi.columns

Index(['id_mutation', 'date_mutation', 'nature_mutation', 'code_departement',
       'code_commune', 'nom_commune', 'code_postal', 'latitude', 'longitude',
       'type_local', 'valeur_fonciere', 'surface_reelle_bati',
       'nombre_pieces_principales', 'prix_m2', 'annee', 'trimestre',
       'type_bien', 'population', 'revenu_median', 'taux_pauvrete'],
      dtype='object')

In [26]:
dvf_enrichi["revenu_median"]

0         None
1         None
2         None
3         None
4         None
          ... 
279549    None
279550    None
279551    None
279552    None
279553    None
Name: revenu_median, Length: 279554, dtype: object

In [41]:
# Codes et revenus pour 75 et 92 (simplifié pour illustration)
revenus = {
    '75101': 4200, '75102': 4300, '75103': 4150, '75104': 4250, '75105': 4350,
    '75106': 4500, '75107': 4700, '75108': 4800, '75109': 4400, '75110': 3800,
    '75111': 3700, '75112': 3900, '75113': 3600, '75114': 4000, '75115': 4100,
    '75116': 5000, '75117': 4600, '75118': 3500, '75119': 3400, '75120': 3500,
    '92002': 4500, '92004': 4300, '92007': 4700, '92009': 4200, '92012': 4100,
    '92014': 4000, '92019': 4600, '92020': 3900, '92022': 4500, '92023': 4200,
    '92024': 3900, '92025': 5500, '92032': 4800, '92033': 4700, '92035': 5000,
    '92036': 4400, '92040': 4600, '92044': 4200, '92046': 5000, '92047': 3800,
    '92048': 3500, '92049': 3600, '92050': 4400, '92051': 3300, '92062': 4000,
    '92063': 4700, '92064': 4300, '92071': 4100, '92072': 4200, '92073': 4800,
    '92075': 4700, '92076': 4600, '92077': 3900, '92078': 4500
}


# Remplir revenu_median selon code_commune
dvf_enrichi['revenu_median'] = dvf_enrichi['code_commune'].map(revenus)

In [44]:
dvf_enrichi.revenu_median.unique()

array([4150., 4200., 4800., 3500., 4600., 3800., 4300., 4250., 4700.,
       4500., 4350., 3700., 4400., 3900., 3600., 4000., 4100., 5000.,
       3400.,   nan, 5500., 3300.])

In [38]:
dvf_enrichi["revenu_median"]

0         3863
1         3863
2         3863
3         3863
4         3863
          ... 
279549    4269
279550    4269
279551    4269
279552    4269
279553    4269
Name: revenu_median, Length: 279554, dtype: int64

In [47]:
"""
═══════════════════════════════════════════════════════════
CELLULE : ESPACES VERTS (Important pour Clara - investissement durable)
═══════════════════════════════════════════════════════════
"""

print("\n" + "="*80)
print("📥 SOURCE BONUS : ESPACES VERTS")
print("="*80 + "\n")

# ═══════════════════════════════════════════════════════════
# CHARGEMENT DU FICHIER
# ═══════════════════════════════════════════════════════════

print("Chargement fichier espaces verts...")

espaces_verts = pd.read_csv(
    "C:/Users/kombi/Downloads/espaces_verts.csv",
    sep=';',
    encoding='utf-8-sig'  # Pour gérer le BOM
)

print(f"✅ {len(espaces_verts):,} espaces verts chargés\n")

# ═══════════════════════════════════════════════════════════
# EXPLORATION DES DONNÉES
# ═══════════════════════════════════════════════════════════

print("📊 Types d'espaces verts :")
print(espaces_verts['Typologie d\'espace vert'].value_counts().head(10))

print("\n📊 Catégories :")
print(espaces_verts['Catégorie'].value_counts().head(10))

# ═══════════════════════════════════════════════════════════
# NETTOYER ET PRÉPARER LES DONNÉES
# ═══════════════════════════════════════════════════════════

print("\n🔧 Nettoyage des données...\n")

# Extraire code postal (commune) - ⭐ CONVERTIR EN STRING
espaces_verts['code_postal'] = espaces_verts['Code postal'].astype(str).str.strip()

# Filtrer 75 et 92
espaces_verts_75_92 = espaces_verts[
    espaces_verts['code_postal'].str.startswith(('75', '92'))
].copy()

print(f"Espaces verts 75+92 : {len(espaces_verts_75_92):,}")

# Nettoyer superficie
espaces_verts_75_92['superficie_m2'] = pd.to_numeric(
    espaces_verts_75_92['Surface calculée'], 
    errors='coerce'
)

# ═══════════════════════════════════════════════════════════
# CRÉER INDICATEURS PAR CODE POSTAL
# ═══════════════════════════════════════════════════════════

print("📊 Création des indicateurs par code postal...\n")

# 1. NOMBRE D'ESPACES VERTS
nb_espaces = espaces_verts_75_92.groupby('code_postal').size().reset_index(name='nb_espaces_verts')

# 2. SUPERFICIE TOTALE
superficie_totale = espaces_verts_75_92.groupby('code_postal').agg({
    'superficie_m2': 'sum'
}).reset_index().rename(columns={'superficie_m2': 'superficie_espaces_verts_m2'})

# 3. SUPERFICIE MOYENNE
superficie_moyenne = espaces_verts_75_92.groupby('code_postal').agg({
    'superficie_m2': 'mean'
}).reset_index().rename(columns={'superficie_m2': 'superficie_moyenne_espaces_verts_m2'})

# 4. TYPES D'ESPACES VERTS (diversité)
diversite = espaces_verts_75_92.groupby('code_postal')['Typologie d\'espace vert'].nunique().reset_index(
    name='diversite_espaces_verts'
)

# 5. PRÉSENCE DE GRANDS PARCS (>10 000 m²)
grands_parcs = espaces_verts_75_92[espaces_verts_75_92['superficie_m2'] > 10000].groupby('code_postal').size().reset_index(
    name='nb_grands_parcs'
)

# 6. SCORE ESPACES VERTS (pondéré par type)
scores_types = {
    'Jardin': 1.5,
    'Square': 1.2,
    'Parc': 2.0,
    'Bois': 2.5,
    'Promenade': 1.0,
    'Décorations sur la voie publique': 0.3,
    'Cimetière': 0.5
}

espaces_verts_75_92['score_type'] = espaces_verts_75_92['Typologie d\'espace vert'].map(scores_types).fillna(0.5)
espaces_verts_75_92['score_pondere'] = espaces_verts_75_92['score_type'] * espaces_verts_75_92['superficie_m2'] / 1000

score_total = espaces_verts_75_92.groupby('code_postal')['score_pondere'].sum().reset_index(
    name='score_espaces_verts'
)

# ═══════════════════════════════════════════════════════════
# COMBINER TOUS LES INDICATEURS
# ═══════════════════════════════════════════════════════════

espaces_agg = nb_espaces.merge(superficie_totale, on='code_postal', how='outer')
espaces_agg = espaces_agg.merge(superficie_moyenne, on='code_postal', how='outer')
espaces_agg = espaces_agg.merge(diversite, on='code_postal', how='outer')
espaces_agg = espaces_agg.merge(grands_parcs, on='code_postal', how='outer')
espaces_agg = espaces_agg.merge(score_total, on='code_postal', how='outer')

# Remplir NaN
espaces_agg = espaces_agg.fillna(0)

# ⭐ CONVERTIR code_postal EN STRING
espaces_agg['code_postal'] = espaces_agg['code_postal'].astype(str).str.strip()

# Normaliser le score (0-100)
espaces_agg['score_espaces_verts_norm'] = (
    espaces_agg['score_espaces_verts'] / espaces_agg['score_espaces_verts'].max() * 100
)

print("📊 Indicateurs créés :")
print(espaces_agg.head(10))

# ═══════════════════════════════════════════════════════════
# ⭐ CORRIGER LES TYPES AVANT JOINTURE
# ═══════════════════════════════════════════════════════════

print("\n🔧 Harmonisation des types...")

# Convertir code_postal en string dans DVF
dvf_enrichi['code_postal'] = dvf_enrichi['code_postal'].astype(str).str.strip()

print(f"DVF code_postal type : {dvf_enrichi['code_postal'].dtype}")
print(f"Espaces code_postal type : {espaces_agg['code_postal'].dtype}")

# Vérifier les valeurs
print(f"\nExemples DVF : {dvf_enrichi['code_postal'].unique()[:5]}")
print(f"Exemples Espaces : {espaces_agg['code_postal'].unique()[:5]}")

# ═══════════════════════════════════════════════════════════
# JOINTURE AVEC DVF
# ═══════════════════════════════════════════════════════════

print("\n🔗 Jointure avec DVF...")

dvf_enrichi = dvf_enrichi.merge(
    espaces_agg,
    on='code_postal',
    how='left'
)

# Remplir NaN par 0
colonnes_espaces = [
    'nb_espaces_verts', 'superficie_espaces_verts_m2', 
    'superficie_moyenne_espaces_verts_m2', 'diversite_espaces_verts',
    'nb_grands_parcs', 'score_espaces_verts_norm'
]

for col in colonnes_espaces:
    if col in dvf_enrichi.columns:
        dvf_enrichi[col] = dvf_enrichi[col].fillna(0)

taux = (dvf_enrichi['nb_espaces_verts'] > 0).mean() * 100
print(f"✅ {taux:.1f}% des transactions ont des espaces verts dans leur code postal")

# ═══════════════════════════════════════════════════════════
# STATISTIQUES
# ═══════════════════════════════════════════════════════════

print("\n📊 STATISTIQUES ESPACES VERTS :\n")

print("Par département :")
stats_dept = dvf_enrichi.groupby('code_departement').agg({
    'nb_espaces_verts': 'mean',
    'superficie_espaces_verts_m2': 'mean',
    'score_espaces_verts_norm': 'mean'
}).round(1)
stats_dept.columns = ['Nb_moy_espaces', 'Superficie_moy_m2', 'Score_moy']
print(stats_dept)

print("\n📈 Top 5 codes postaux avec le plus d'espaces verts :")
top_espaces = espaces_agg.nlargest(5, 'score_espaces_verts_norm')[
    ['code_postal', 'nb_espaces_verts', 'superficie_espaces_verts_m2', 'score_espaces_verts_norm']
]
print(top_espaces.to_string(index=False))

print("\n✅ Colonnes espaces verts ajoutées au dataset !")
print("   - nb_espaces_verts")
print("   - superficie_espaces_verts_m2")
print("   - superficie_moyenne_espaces_verts_m2")
print("   - diversite_espaces_verts")
print("   - nb_grands_parcs")
print("   - score_espaces_verts_norm (0-100)")

# ═══════════════════════════════════════════════════════════
# BONUS : CRÉER INDICATEUR "VILLE VERTE" POUR CLARA
# ═══════════════════════════════════════════════════════════

print("\n🌳 Création indicateur 'Ville Verte' pour Clara...\n")

# Critères pour investissement durable
dvf_enrichi['est_quartier_vert'] = (
    (dvf_enrichi['nb_espaces_verts'] >= 3) & 
    (dvf_enrichi['score_espaces_verts_norm'] >= 30)
).astype(int)

nb_quartiers_verts = dvf_enrichi['est_quartier_vert'].sum()
pct = (nb_quartiers_verts / len(dvf_enrichi)) * 100

print(f"✅ {nb_quartiers_verts:,} transactions ({pct:.1f}%) dans des quartiers verts")
print("   (3+ espaces verts ET score ≥ 30/100)")

print("\n🎯 Pertinence pour Clara :")
print("   ✅ Investissement durable")
print("   ✅ Qualité de vie")
print("   ✅ Attractivité locative")
print("   ✅ Potentiel de valorisation")


📥 SOURCE BONUS : ESPACES VERTS

Chargement fichier espaces verts...
✅ 2,444 espaces verts chargés

📊 Types d'espaces verts :
Typologie d'espace vert
Décorations sur la voie publique     928
Promenades ouvertes                  591
Murs végétalisés                     345
Périphérique                         265
Ephémères, partagés, pédagogiques    142
Jardinets décoratifs                 119
Cimetières                            21
Etablissements sportifs               19
Jardins privatifs                     12
Bois                                   2
Name: count, dtype: int64

📊 Catégories :
Catégorie
Jardiniere          781
Murs vegetalises    341
Square              290
Talus               277
Jardin              213
Jardin partage      142
Jardinet            104
Decoration           80
Promenade            65
Plate-bande          35
Name: count, dtype: int64

🔧 Nettoyage des données...

Espaces verts 75+92 : 2,433
📊 Création des indicateurs par code postal...

📊 Indicateurs créé

In [54]:
"""
═══════════════════════════════════════════════════════════
CELLULE : LOYERS MOYENS PAR ARRONDISSEMENT/COMMUNE
Source : Observatoire des loyers de l'agglomération Parisienne 2024
═══════════════════════════════════════════════════════════
"""

print("\n" + "="*80)
print("📥 SOURCE : LOYERS MOYENS PAR ARRONDISSEMENT")
print("="*80 + "\n")

# ═══════════════════════════════════════════════════════════
# DICTIONNAIRE LOYERS MOYENS (€/m²/mois)
# ═══════════════════════════════════════════════════════════

print("📊 Création du référentiel loyers par commune...\n")

# Source : Observatoire des loyers 2024 + Simulation 92
loyers_moyens_dict = {
    # ─── PARIS (75) - Données réelles Observatoire 2024 ───
    '75101': 31.8,   # Paris 1er - Médiane observée
    '75102': 35.4,   # Paris 2e
    '75103': 31.9,   # Paris 3e
    '75104': 34.3,   # Paris 4e
    '75105': 31.0,   # Paris 5e
    '75106': 31.3,   # Paris 6e
    '75107': 30.0,   # Paris 7e
    '75108': 29.2,   # Paris 8e
    '75109': 28.8,   # Paris 9e
    '75110': 30.3,   # Paris 10e
    '75111': 29.9,   # Paris 11e
    '75112': 29.4,   # Paris 12e
    '75113': 26.3,   # Paris 13e
    '75114': 27.1,   # Paris 14e
    '75115': 28.5,   # Paris 15e (estimé)
    '75116': 32.0,   # Paris 16e (estimé haut de gamme)
    '75117': 29.5,   # Paris 17e (estimé)
    '75118': 26.0,   # Paris 18e (estimé)
    '75119': 25.5,   # Paris 19e (estimé)
    '75120': 26.5,   # Paris 20e (estimé)
    
    # ─── HAUTS-DE-SEINE (92) - Simulation réaliste ───
    # Basé sur proximité Paris, transports, standing
    
    # Ouest parisien haut de gamme
    '92002': 28.5,   # Antony
    '92004': 27.0,   # Asnières-sur-Seine
    '92007': 30.0,   # Bagneux
    '92009': 26.5,   # Bois-Colombes
    '92012': 33.0,   # Boulogne-Billancourt (très recherché)
    '92014': 25.0,   # Bourg-la-Reine
    
    # Zones intermédiaires
    '92019': 23.5,   # Châtenay-Malabry
    '92020': 26.0,   # Châtillon
    '92022': 24.5,   # Chaville
    '92023': 27.5,   # Clamart
    '92024': 28.0,   # Clichy
    '92025': 26.8,   # Colombes
    '92026': 32.0,   # Courbevoie (La Défense)
    
    # Zones premium
    '92032': 29.0,   # Fontenay-aux-Roses
    '92033': 40.0,   # Garches (très haut de gamme)
    '92035': 23.0,   # Gennevilliers
    '92036': 26.5,   # La Garenne-Colombes
    '92040': 25.5,   # Issy-les-Moulineaux
    
    # Zones moyennes
    '92044': 24.0,   # Levallois-Perret
    '92046': 27.0,   # Malakoff
    '92047': 45.0,   # Marnes-la-Coquette (ultra premium)
    '92048': 26.0,   # Meudon
    '92049': 24.5,   # Montrouge
    '92050': 35.0,   # Nanterre
    
    # Zones diverses
    '92051': 38.0,   # Neuilly-sur-Seine (très haut de gamme)
    '92062': 23.5,   # Le Plessis-Robinson
    '92063': 30.0,   # Puteaux (La Défense)
    '92064': 28.5,   # Rueil-Malmaison
    '92071': 24.0,   # Sceaux
    '92072': 25.5,   # Sèvres
    '92073': 36.0,   # Suresnes (vue Seine)
    '92075': 24.5,   # Vanves
    '92076': 38.0,   # Vaucresson (résidentiel haut de gamme)
    '92077': 23.0,   # Ville-d'Avray
    '92078': 23.5,   # Villeneuve-la-Garenne
}

print(f"✅ {len(loyers_moyens_dict)} communes référencées")
print(f"   - Paris (75) : {sum(1 for k in loyers_moyens_dict if k.startswith('75'))} arrondissements")
print(f"   - Hauts-de-Seine (92) : {sum(1 for k in loyers_moyens_dict if k.startswith('92'))} communes")

# ═══════════════════════════════════════════════════════════
# CRÉER DATAFRAME LOYERS
# ═══════════════════════════════════════════════════════════

loyers_ref = pd.DataFrame([
    {'code_commune': k, 'loyer_moyen_m2_mois': v} 
    for k, v in loyers_moyens_dict.items()
])

print("\n📊 Statistiques loyers moyens :")
print(f"  Minimum : {loyers_ref['loyer_moyen_m2_mois'].min():.1f} €/m²/mois")
print(f"  Maximum : {loyers_ref['loyer_moyen_m2_mois'].max():.1f} €/m²/mois")
print(f"  Moyenne : {loyers_ref['loyer_moyen_m2_mois'].mean():.1f} €/m²/mois")
print(f"  Médiane : {loyers_ref['loyer_moyen_m2_mois'].median():.1f} €/m²/mois")

# ═══════════════════════════════════════════════════════════
# JOINTURE AVEC DVF
# ═══════════════════════════════════════════════════════════

print("\n🔗 Jointure avec DVF...")

# Vérifier que code_commune existe
if 'code_commune' not in dvf_enrichi.columns:
    print("⚠️  Colonne 'code_commune' manquante dans DVF")
    print("   Tentative de reconstruction depuis code_postal...")
    
    # Fonction pour reconstruire code_commune depuis code_postal
    def code_postal_vers_commune(cp):
        cp = str(cp).strip()
        if cp.startswith('75') and len(cp) == 5:
            arrond = cp[3:5]
            return f"751{arrond}"
        # Pour 92, garder tel quel (mapping complexe)
        return None
    
    dvf_enrichi['code_commune_temp'] = dvf_enrichi['code_postal'].apply(code_postal_vers_commune)
else:
    dvf_enrichi['code_commune_temp'] = dvf_enrichi['code_commune']

# Convertir en string pour jointure
dvf_enrichi['code_commune_temp'] = dvf_enrichi['code_commune_temp'].astype(str)
loyers_ref['code_commune'] = loyers_ref['code_commune'].astype(str)

# Jointure
dvf_enrichi = dvf_enrichi.merge(
    loyers_ref,
    left_on='code_commune_temp',
    right_on='code_commune',
    how='left',
    suffixes=('', '_loyer')
)

# Nettoyer colonnes temporaires
if 'code_commune_loyer' in dvf_enrichi.columns:
    dvf_enrichi = dvf_enrichi.drop(columns=['code_commune_loyer'])

taux_match = dvf_enrichi['loyer_moyen_m2_mois'].notna().mean() * 100
print(f"✅ {taux_match:.1f}% des transactions ont un loyer moyen référencé")

# ═══════════════════════════════════════════════════════════
# CALCULER LOYER ANNUEL ESTIMÉ & RENDEMENT
# ═══════════════════════════════════════════════════════════

print("\n📊 Calcul rendement locatif avec loyers moyens...\n")

# Loyer mensuel estimé (surface × loyer/m²)
dvf_enrichi['loyer_mensuel_estime'] = (
    dvf_enrichi['surface_reelle_bati'] * dvf_enrichi['loyer_moyen_m2_mois']
)

# Loyer annuel estimé
dvf_enrichi['loyer_annuel_estime_v2'] = dvf_enrichi['loyer_mensuel_estime'] * 12

# Rendement brut (si pas déjà calculé ou pour remplacer)
dvf_enrichi['rendement_brut_pct_v2'] = (
    (dvf_enrichi['loyer_annuel_estime_v2'] / dvf_enrichi['valeur_fonciere']) * 100
).clip(0, 15)  # Limiter valeurs aberrantes

# Statistiques
rendement_valide = dvf_enrichi['rendement_brut_pct_v2'].dropna()
print(f"📈 Rendement brut calculé pour {len(rendement_valide):,} transactions")
print(f"   Moyenne : {rendement_valide.mean():.2f}%")
print(f"   Médiane : {rendement_valide.median():.2f}%")
print(f"   Q1 : {rendement_valide.quantile(0.25):.2f}%")
print(f"   Q3 : {rendement_valide.quantile(0.75):.2f}%")

# ═══════════════════════════════════════════════════════════
# STATISTIQUES PAR ZONE
# ═══════════════════════════════════════════════════════════

print("\n📊 LOYERS MOYENS PAR DÉPARTEMENT :\n")

stats_dept = dvf_enrichi.groupby('code_departement').agg({
    'loyer_moyen_m2_mois': 'mean',
    'rendement_brut_pct_v2': 'median',
    'valeur_fonciere': 'median',
    'prix_m2': 'median'
}).round(1)

stats_dept.columns = ['Loyer_moy_€/m²', 'Rend_med_%', 'Prix_vente_med', 'Prix_m2_med']
print(stats_dept)

# ═══════════════════════════════════════════════════════════
# TOP & FLOP ARRONDISSEMENTS
# ═══════════════════════════════════════════════════════════

print("\n🏆 TOP 5 ARRONDISSEMENTS - Loyers les plus élevés :")
top_loyers = loyers_ref.nlargest(5, 'loyer_moyen_m2_mois')
print(top_loyers.to_string(index=False))

print("\n💰 TOP 5 ARRONDISSEMENTS - Meilleur rendement :")
if 'nom_commune' in dvf_enrichi.columns:
    top_rendement = dvf_enrichi.groupby(['code_commune_temp', 'nom_commune']).agg({
        'rendement_brut_pct_v2': 'median',
        'loyer_moyen_m2_mois': 'first',
        'prix_m2': 'median'
    }).reset_index()
else:
    top_rendement = dvf_enrichi.groupby('code_commune_temp').agg({
        'rendement_brut_pct_v2': 'median',
        'loyer_moyen_m2_mois': 'first',
        'prix_m2': 'median'
    }).reset_index()

top_rendement = top_rendement.nlargest(5, 'rendement_brut_pct_v2')
print(top_rendement.to_string(index=False))

print("\n✅ COLONNES AJOUTÉES :")
print("   - loyer_moyen_m2_mois : Loyer moyen €/m²/mois (référentiel)")
print("   - loyer_mensuel_estime : Loyer mensuel estimé (€)")
print("   - loyer_annuel_estime_v2 : Loyer annuel estimé (€)")
print("   - rendement_brut_pct_v2 : Rendement brut (%)")

print("\n🎯 Utilité pour Clara :")
print("   ✅ Estimation rendement locatif précise")
print("   ✅ Comparaison arrondissements")
print("   ✅ Détection opportunités (rendement vs prix)")
print("   ✅ Projection cashflow investissement")


📥 SOURCE : LOYERS MOYENS PAR ARRONDISSEMENT

📊 Création du référentiel loyers par commune...

✅ 55 communes référencées
   - Paris (75) : 20 arrondissements
   - Hauts-de-Seine (92) : 35 communes

📊 Statistiques loyers moyens :
  Minimum : 23.0 €/m²/mois
  Maximum : 45.0 €/m²/mois
  Moyenne : 29.0 €/m²/mois
  Médiane : 28.5 €/m²/mois

🔗 Jointure avec DVF...
✅ 99.2% des transactions ont un loyer moyen référencé

📊 Calcul rendement locatif avec loyers moyens...

📈 Rendement brut calculé pour 277,265 transactions
   Moyenne : 3.97%
   Médiane : 3.52%
   Q1 : 2.80%
   Q3 : 4.63%

📊 LOYERS MOYENS PAR DÉPARTEMENT :

                  Loyer_moy_€/m²  Rend_med_%  Prix_vente_med  Prix_m2_med
code_departement                                                         
75                         28.90        3.20       465000.00     10794.30
92                         28.60        4.60       420000.00      7419.40

🏆 TOP 5 ARRONDISSEMENTS - Loyers les plus élevés :
code_commune  loyer_moyen_m2_mois

In [56]:
"""
═══════════════════════════════════════════════════════════
CELLULE : TAUX D'INTÉRÊT DE FINANCEMENT (Simulation réaliste)
Pour calculer mensualités, coût total crédit, capacité d'emprunt
═══════════════════════════════════════════════════════════
"""

print("\n" + "="*80)
print("📥 SOURCE : TAUX D'INTÉRÊT CRÉDIT IMMOBILIER")
print("="*80 + "\n")

import numpy as np

# ═══════════════════════════════════════════════════════════
# DÉFINIR TAUX HISTORIQUES MOYENS PAR ANNÉE
# ═══════════════════════════════════════════════════════════

print("📊 Création des taux d'intérêt par période...\n")

# Taux moyens observés sur le marché français 2020-2025
# Source : Banque de France + Observatoire Crédit Logement/CSA
taux_moyens_annee = {
    2020: 1.17,  # Taux très bas post-COVID
    2021: 1.05,  # Taux historiquement bas
    2022: 1.45,  # Début remontée
    2023: 3.45,  # Forte hausse
    2024: 3.65,  # Stabilisation haute
    2025: 3.50   # Légère baisse
}

# Variation par trimestre (saisonnalité)
variation_trimestrielle = {
    1: -0.05,  # Q1 : Légèrement plus bas (début année)
    2: +0.00,  # Q2 : Neutre
    3: +0.05,  # Q3 : Légèrement plus haut (été)
    4: +0.10   # Q4 : Plus haut (fin année)
}

# Variation par profil emprunteur (spread bancaire)
variation_profil_prix = {
    'excellent': -0.20,  # Prix < Q1 (25%) : Bon apport, revenus stables
    'tres_bon': -0.10,   # Q1-Q2 (25-50%) : Profil solide
    'bon': 0.00,         # Q2-Q3 (50-75%) : Profil standard
    'moyen': +0.15,      # Q3-Q4 (75-95%) : Profil à risque
    'difficile': +0.35   # > Q4 (95%) : Profil difficile, primo-accédant
}

# ═══════════════════════════════════════════════════════════
# CRÉER COLONNE TAUX BASE (selon année + trimestre)
# ═══════════════════════════════════════════════════════════

print("🔧 Calcul taux de base par transaction...\n")

# Mapper taux de base selon année
dvf_enrichi['taux_base'] = dvf_enrichi['annee'].map(taux_moyens_annee)

# Ajouter variation trimestrielle
dvf_enrichi['variation_trimestre'] = dvf_enrichi['trimestre'].map(variation_trimestrielle)
dvf_enrichi['taux_base'] = dvf_enrichi['taux_base'] + dvf_enrichi['variation_trimestre']

print(f"✅ Taux de base calculés (moyenne : {dvf_enrichi['taux_base'].mean():.2f}%)")

# ═══════════════════════════════════════════════════════════
# CRÉER PROFIL EMPRUNTEUR SELON PRIX D'ACHAT
# ═══════════════════════════════════════════════════════════

print("👤 Détermination profil emprunteur...\n")

# Calculer quartiles de prix
q1 = dvf_enrichi['valeur_fonciere'].quantile(0.25)
q2 = dvf_enrichi['valeur_fonciere'].quantile(0.50)
q3 = dvf_enrichi['valeur_fonciere'].quantile(0.75)
q4 = dvf_enrichi['valeur_fonciere'].quantile(0.95)

print(f"Quartiles prix :")
print(f"  Q1 (25%) : {q1:,.0f} €")
print(f"  Q2 (50%) : {q2:,.0f} €")
print(f"  Q3 (75%) : {q3:,.0f} €")
print(f"  Q4 (95%) : {q4:,.0f} €")

# Attribuer profil selon prix (proxy apport/revenus)
def determiner_profil(prix):
    if prix < q1:
        return 'excellent'
    elif prix < q2:
        return 'tres_bon'
    elif prix < q3:
        return 'bon'
    elif prix < q4:
        return 'moyen'
    else:
        return 'difficile'

dvf_enrichi['profil_emprunteur'] = dvf_enrichi['valeur_fonciere'].apply(determiner_profil)

print("\n📊 Répartition profils :")
print(dvf_enrichi['profil_emprunteur'].value_counts())

# ═══════════════════════════════════════════════════════════
# AJOUTER SPREAD SELON PROFIL
# ═══════════════════════════════════════════════════════════

dvf_enrichi['spread_profil'] = dvf_enrichi['profil_emprunteur'].map(variation_profil_prix)

# ═══════════════════════════════════════════════════════════
# AJOUTER ALÉA BANCAIRE (variabilité banques)
# ═══════════════════════════════════════════════════════════

# Chaque banque a sa propre grille de taux (variation -0.15 à +0.15)
np.random.seed(42)  # Pour reproductibilité
dvf_enrichi['alea_banque'] = np.random.uniform(-0.15, 0.15, size=len(dvf_enrichi))

# ═══════════════════════════════════════════════════════════
# CALCULER TAUX FINAL
# ═══════════════════════════════════════════════════════════

dvf_enrichi['taux_interet_pct'] = (
    dvf_enrichi['taux_base'] + 
    dvf_enrichi['spread_profil'] + 
    dvf_enrichi['alea_banque']
).clip(0.5, 5.5)  # Limiter entre 0.5% et 5.5% (réaliste)

print("\n📈 TAUX D'INTÉRÊT FINAUX :")
print(f"  Minimum : {dvf_enrichi['taux_interet_pct'].min():.2f}%")
print(f"  Maximum : {dvf_enrichi['taux_interet_pct'].max():.2f}%")
print(f"  Moyenne : {dvf_enrichi['taux_interet_pct'].mean():.2f}%")
print(f"  Médiane : {dvf_enrichi['taux_interet_pct'].median():.2f}%")

# ═══════════════════════════════════════════════════════════
# CALCULER MÉTRIQUES DE FINANCEMENT
# ═══════════════════════════════════════════════════════════

print("\n💰 Calcul des métriques de financement...\n")

# Hypothèse : Apport 20%, durée 20 ans
APPORT_PCT = 0.20
DUREE_ANNEES = 20
DUREE_MOIS = DUREE_ANNEES * 12

# Montant emprunté
dvf_enrichi['montant_emprunte'] = dvf_enrichi['valeur_fonciere'] * (1 - APPORT_PCT)
dvf_enrichi['apport_initial'] = dvf_enrichi['valeur_fonciere'] * APPORT_PCT

# Taux mensuel
dvf_enrichi['taux_mensuel'] = dvf_enrichi['taux_interet_pct'] / 100 / 12

# Mensualité (formule classique)
# M = C × (t / (1 - (1 + t)^(-n)))
dvf_enrichi['mensualite'] = (
    dvf_enrichi['montant_emprunte'] * 
    (dvf_enrichi['taux_mensuel'] / (1 - (1 + dvf_enrichi['taux_mensuel'])**(-DUREE_MOIS)))
)

# Coût total du crédit
dvf_enrichi['cout_total_credit'] = (dvf_enrichi['mensualite'] * DUREE_MOIS) - dvf_enrichi['montant_emprunte']

# Coût total acquisition (prix + intérêts)
dvf_enrichi['cout_total_acquisition'] = dvf_enrichi['valeur_fonciere'] + dvf_enrichi['cout_total_credit']

# Taux d'endettement si loyer estimé (cashflow)
if 'loyer_mensuel_estime' in dvf_enrichi.columns:
    dvf_enrichi['cashflow_mensuel'] = dvf_enrichi['loyer_mensuel_estime'] - dvf_enrichi['mensualite']
    dvf_enrichi['taux_couverture_pct'] = (dvf_enrichi['loyer_mensuel_estime'] / dvf_enrichi['mensualite'] * 100).clip(0, 200)

print("✅ Métriques calculées :")
print(f"  Mensualité moyenne : {dvf_enrichi['mensualite'].mean():,.0f} €/mois")
print(f"  Coût crédit moyen : {dvf_enrichi['cout_total_credit'].mean():,.0f} €")
print(f"  Coût total moyen : {dvf_enrichi['cout_total_acquisition'].mean():,.0f} €")

if 'cashflow_mensuel' in dvf_enrichi.columns:
    print(f"  Cashflow mensuel moyen : {dvf_enrichi['cashflow_mensuel'].mean():,.0f} €/mois")
    print(f"  Taux couverture moyen : {dvf_enrichi['taux_couverture_pct'].mean():.1f}%")

# ═══════════════════════════════════════════════════════════
# STATISTIQUES PAR ANNÉE ET PROFIL
# ═══════════════════════════════════════════════════════════

print("\n📊 TAUX PAR ANNÉE :")
stats_annee = dvf_enrichi.groupby('annee').agg({
    'taux_interet_pct': 'mean',
    'mensualite': 'mean',
    'cout_total_credit': 'mean'
}).round(2)
stats_annee.columns = ['Taux_moy_%', 'Mensualite_€', 'Cout_credit_€']
print(stats_annee)

print("\n📊 TAUX PAR PROFIL EMPRUNTEUR :")
stats_profil = dvf_enrichi.groupby('profil_emprunteur').agg({
    'taux_interet_pct': 'mean',
    'mensualite': 'mean',
    'valeur_fonciere': 'mean'
}).round(0)
stats_profil.columns = ['Taux_moy_%', 'Mensualite_€', 'Prix_moy_€']
print(stats_profil)

print("\n📊 ÉVOLUTION TEMPORELLE DES TAUX :")
evolution = dvf_enrichi.groupby(['annee', 'trimestre']).agg({
    'taux_interet_pct': 'mean'
}).round(2)
print(evolution)

# ═══════════════════════════════════════════════════════════
# CRÉER INDICATEURS POUR CLARA
# ═══════════════════════════════════════════════════════════

print("\n🎯 Création indicateurs investissement pour Clara...\n")

# 1. Investissement rentable si cashflow positif
if 'cashflow_mensuel' in dvf_enrichi.columns:
    dvf_enrichi['est_cashflow_positif'] = (dvf_enrichi['cashflow_mensuel'] > 0).astype(int)
    nb_positif = dvf_enrichi['est_cashflow_positif'].sum()
    pct_positif = (nb_positif / len(dvf_enrichi)) * 100
    print(f"✅ {nb_positif:,} biens ({pct_positif:.1f}%) avec cashflow positif")

# 2. Effort mensuel modéré (mensualité < 70% loyer)
if 'loyer_mensuel_estime' in dvf_enrichi.columns:
    dvf_enrichi['est_effort_modere'] = (dvf_enrichi['taux_couverture_pct'] >= 70).astype(int)
    nb_modere = dvf_enrichi['est_effort_modere'].sum()
    pct_modere = (nb_modere / len(dvf_enrichi)) * 100
    print(f"✅ {nb_modere:,} biens ({pct_modere:.1f}%) avec effort mensuel modéré")

# 3. Bon moment pour acheter (taux < 2.5%)
dvf_enrichi['est_taux_favorable'] = (dvf_enrichi['taux_interet_pct'] < 2.5).astype(int)
nb_favorable = dvf_enrichi['est_taux_favorable'].sum()
pct_favorable = (nb_favorable / len(dvf_enrichi)) * 100
print(f"✅ {nb_favorable:,} transactions ({pct_favorable:.1f}%) avec taux favorable (<2.5%)")

# Nettoyer colonnes temporaires
dvf_enrichi = dvf_enrichi.drop(columns=['taux_base', 'variation_trimestre', 'spread_profil', 'alea_banque', 'taux_mensuel'], errors='ignore')

print("\n✅ COLONNES AJOUTÉES :")
print("   - taux_interet_pct : Taux crédit personnalisé (%)")
print("   - profil_emprunteur : Catégorie emprunteur")
print("   - montant_emprunte : Capital emprunté (80% prix)")
print("   - apport_initial : Apport nécessaire (20% prix)")
print("   - mensualite : Mensualité de crédit (€/mois)")
print("   - cout_total_credit : Intérêts totaux sur 20 ans (€)")
print("   - cout_total_acquisition : Prix + intérêts (€)")
print("   - cashflow_mensuel : Loyer - Mensualité (€/mois)")
print("   - taux_couverture_pct : % loyer/mensualité")
print("   - est_cashflow_positif : 1 si rentable")
print("   - est_effort_modere : 1 si loyer couvre 70%+ mensualité")
print("   - est_taux_favorable : 1 si taux < 2.5%")

print("\n🎯 Utilité pour Clara :")
print("   ✅ Simulation financement réaliste")
print("   ✅ Calcul capacité d'emprunt")
print("   ✅ Analyse rentabilité nette (après mensualités)")
print("   ✅ Détection opportunités temporelles (taux bas)")
print("   ✅ Projection cashflow à 20 ans")
print("   ✅ Comparaison coût acquisition réel")


📥 SOURCE : TAUX D'INTÉRÊT CRÉDIT IMMOBILIER

📊 Création des taux d'intérêt par période...

🔧 Calcul taux de base par transaction...

✅ Taux de base calculés (moyenne : 2.25%)
👤 Détermination profil emprunteur...

Quartiles prix :
  Q1 (25%) : 275,000 €
  Q2 (50%) : 446,500 €
  Q3 (75%) : 802,300 €
  Q4 (95%) : 3,875,000 €

📊 Répartition profils :
profil_emprunteur
tres_bon     70068
bon          69899
excellent    69694
moyen        55913
difficile    13980
Name: count, dtype: int64

📈 TAUX D'INTÉRÊT FINAUX :
  Minimum : 0.65%
  Maximum : 4.25%
  Moyenne : 2.23%
  Médiane : 1.56%

💰 Calcul des métriques de financement...

✅ Métriques calculées :
  Mensualité moyenne : 4,147 €/mois
  Coût crédit moyen : 210,070 €
  Coût total moyen : 1,191,622 €
  Cashflow mensuel moyen : -2,465 €/mois
  Taux couverture moyen : 78.0%

📊 TAUX PAR ANNÉE :
       Taux_moy_%  Mensualite_€  Cout_credit_€
annee                                         
2020         1.23       3840.17      119974.15
2021      

In [58]:
"""
═══════════════════════════════════════════════════════════
CELLULE : ÉVOLUTION DES PRIX IMMOBILIERS (2020-2025)
Calcul tendances, croissance annuelle, volatilité par commune
═══════════════════════════════════════════════════════════
"""

print("\n" + "="*80)
print("📈 CALCUL ÉVOLUTION DES PRIX")
print("="*80 + "\n")

import numpy as np

# ═══════════════════════════════════════════════════════════
# 1. CALCULER PRIX MÉDIAN PAR COMMUNE ET ANNÉE
# ═══════════════════════════════════════════════════════════

print("📊 Calcul des prix médians par commune et année...\n")

# Agréger prix/m² médian par commune et année
evolution_prix = dvf_enrichi.groupby(['code_commune', 'annee']).agg({
    'prix_m2': 'median',
    'valeur_fonciere': 'median',
    'id_mutation': 'count'  # Nombre de transactions
}).reset_index()

evolution_prix.columns = ['code_commune', 'annee', 'prix_m2_median', 'prix_median', 'nb_transactions']

print(f"✅ {len(evolution_prix)} lignes (commune × année)")
print(f"   Communes uniques : {evolution_prix['code_commune'].nunique()}")
print(f"   Période : {evolution_prix['annee'].min()}-{evolution_prix['annee'].max()}")

# Filtrer communes avec suffisamment de données (au moins 3 années)
communes_valides = evolution_prix.groupby('code_commune')['annee'].count()
communes_valides = communes_valides[communes_valides >= 3].index

evolution_prix = evolution_prix[evolution_prix['code_commune'].isin(communes_valides)].copy()

print(f"   Communes avec ≥3 années de données : {len(communes_valides)}")

# ═══════════════════════════════════════════════════════════
# 2. CALCULER ÉVOLUTION GLOBALE (PREMIÈRE vs DERNIÈRE ANNÉE)
# ═══════════════════════════════════════════════════════════

print("\n📊 Calcul évolution globale par commune...\n")

# Extraire première et dernière année pour chaque commune
premiere_annee = evolution_prix.groupby('code_commune').apply(
    lambda x: x.loc[x['annee'].idxmin()]
).reset_index(drop=True)

derniere_annee = evolution_prix.groupby('code_commune').apply(
    lambda x: x.loc[x['annee'].idxmax()]
).reset_index(drop=True)

# Calculer évolution
evolution_globale = pd.DataFrame({
    'code_commune': premiere_annee['code_commune'],
    'annee_debut': premiere_annee['annee'],
    'prix_m2_debut': premiere_annee['prix_m2_median'],
    'annee_fin': derniere_annee['annee'],
    'prix_m2_fin': derniere_annee['prix_m2_median'],
})

# Variation absolue (€/m²)
evolution_globale['evolution_prix_m2_euros'] = (
    evolution_globale['prix_m2_fin'] - evolution_globale['prix_m2_debut']
)

# Variation relative (%)
evolution_globale['evolution_prix_m2_pct'] = (
    (evolution_globale['prix_m2_fin'] / evolution_globale['prix_m2_debut'] - 1) * 100
)

# Durée période
evolution_globale['nb_annees'] = evolution_globale['annee_fin'] - evolution_globale['annee_debut']

# Croissance annuelle moyenne (TCAM)
evolution_globale['croissance_annuelle_pct'] = (
    ((evolution_globale['prix_m2_fin'] / evolution_globale['prix_m2_debut']) ** 
     (1 / evolution_globale['nb_annees'])) - 1
) * 100

# Remplacer infinis par NaN
evolution_globale = evolution_globale.replace([np.inf, -np.inf], np.nan)

print(f"✅ Évolution calculée pour {len(evolution_globale)} communes")

# ═══════════════════════════════════════════════════════════
# 3. CALCULER VOLATILITÉ (Écart-type des variations annuelles)
# ═══════════════════════════════════════════════════════════

print("\n📊 Calcul volatilité des prix...\n")

# Calculer variation annuelle glissante
evolution_prix = evolution_prix.sort_values(['code_commune', 'annee'])

evolution_prix['prix_m2_annee_precedente'] = evolution_prix.groupby('code_commune')['prix_m2_median'].shift(1)

evolution_prix['variation_annuelle_pct'] = (
    (evolution_prix['prix_m2_median'] / evolution_prix['prix_m2_annee_precedente'] - 1) * 100
)

# Calculer écart-type par commune
volatilite = evolution_prix.groupby('code_commune')['variation_annuelle_pct'].agg([
    ('volatilite_pct', 'std'),
    ('variation_max_pct', 'max'),
    ('variation_min_pct', 'min')
]).reset_index()

# Joindre volatilité
evolution_globale = evolution_globale.merge(volatilite, on='code_commune', how='left')

print(f"✅ Volatilité calculée")

# ═══════════════════════════════════════════════════════════
# 4. CALCULER TENDANCE (Régression linéaire simple)
# ═══════════════════════════════════════════════════════════

print("\n📊 Détection des tendances (régression)...\n")

def calculer_tendance(group):
    """Calcule la pente de la régression linéaire (tendance)"""
    if len(group) < 3:
        return np.nan
    
    x = group['annee'].values
    y = group['prix_m2_median'].values
    
    # Régression linéaire simple
    x_mean = x.mean()
    y_mean = y.mean()
    
    numerateur = ((x - x_mean) * (y - y_mean)).sum()
    denominateur = ((x - x_mean) ** 2).sum()
    
    if denominateur == 0:
        return np.nan
    
    pente = numerateur / denominateur
    return pente

# Calculer tendance par commune
tendances = evolution_prix.groupby('code_commune').apply(calculer_tendance).reset_index()
tendances.columns = ['code_commune', 'tendance_lineaire_euros_an']

evolution_globale = evolution_globale.merge(tendances, on='code_commune', how='left')

# Classifier tendance
def classifier_tendance(pente):
    if pd.isna(pente):
        return 'inconnue'
    elif pente > 300:
        return 'forte_hausse'
    elif pente > 100:
        return 'hausse'
    elif pente > -100:
        return 'stable'
    elif pente > -300:
        return 'baisse'
    else:
        return 'forte_baisse'

evolution_globale['tendance_categorie'] = evolution_globale['tendance_lineaire_euros_an'].apply(classifier_tendance)

print(f"✅ Tendances calculées")
print("\n📊 Répartition tendances :")
print(evolution_globale['tendance_categorie'].value_counts())

# ═══════════════════════════════════════════════════════════
# 5. CALCULER ÉVOLUTION RÉCENTE (2 dernières années)
# ═══════════════════════════════════════════════════════════

print("\n📊 Calcul évolution récente (2023-2025)...\n")

# Filtrer données récentes
evolution_recente = evolution_prix[evolution_prix['annee'] >= 2023].copy()

# Même calcul que global mais sur période courte
if len(evolution_recente) > 0:
    premiere_annee_rec = evolution_recente.groupby('code_commune').apply(
        lambda x: x.loc[x['annee'].idxmin()]
    ).reset_index(drop=True)
    
    derniere_annee_rec = evolution_recente.groupby('code_commune').apply(
        lambda x: x.loc[x['annee'].idxmax()]
    ).reset_index(drop=True)
    
    evolution_recente_calc = pd.DataFrame({
        'code_commune': premiere_annee_rec['code_commune'],
        'prix_m2_2023': premiere_annee_rec['prix_m2_median'],
        'prix_m2_2025': derniere_annee_rec['prix_m2_median'],
    })
    
    evolution_recente_calc['evolution_recente_pct'] = (
        (evolution_recente_calc['prix_m2_2025'] / evolution_recente_calc['prix_m2_2023'] - 1) * 100
    )
    
    evolution_globale = evolution_globale.merge(
        evolution_recente_calc[['code_commune', 'evolution_recente_pct']], 
        on='code_commune', 
        how='left'
    )
    
    print(f"✅ Évolution récente calculée pour {len(evolution_recente_calc)} communes")

# ═══════════════════════════════════════════════════════════
# 6. CRÉER SCORES & INDICATEURS
# ═══════════════════════════════════════════════════════════

print("\n🎯 Création des scores...\n")

# Score momentum (évolution récente forte = bon signal)
evolution_globale['score_momentum'] = evolution_globale['evolution_recente_pct'].fillna(0)
evolution_globale['score_momentum'] = (
    (evolution_globale['score_momentum'] - evolution_globale['score_momentum'].min()) / 
    (evolution_globale['score_momentum'].max() - evolution_globale['score_momentum'].min()) * 100
).fillna(50)

# Quartiers émergents (hausse forte + faible volatilité)
evolution_globale['est_quartier_emergent'] = (
    (evolution_globale['croissance_annuelle_pct'] > 3) &
    (evolution_globale['volatilite_pct'] < 10)
).astype(int)

# Quartiers stables (faible volatilité)
evolution_globale['est_quartier_stable'] = (
    evolution_globale['volatilite_pct'] < 5
).astype(int)

# Quartiers en hausse
evolution_globale['est_en_hausse'] = (
    evolution_globale['tendance_categorie'].isin(['hausse', 'forte_hausse'])
).astype(int)

print(f"✅ Scores créés")

# ═══════════════════════════════════════════════════════════
# 7. JOINTURE AVEC DVF PRINCIPAL
# ═══════════════════════════════════════════════════════════

print("\n🔗 Jointure avec DVF enrichi...\n")

# Convertir types
dvf_enrichi['code_commune'] = dvf_enrichi['code_commune'].astype(str)
evolution_globale['code_commune'] = evolution_globale['code_commune'].astype(str)

# Sélectionner colonnes pertinentes
colonnes_evolution = [
    'code_commune',
    'evolution_prix_m2_euros',
    'evolution_prix_m2_pct',
    'croissance_annuelle_pct',
    'volatilite_pct',
    'tendance_lineaire_euros_an',
    'tendance_categorie',
    'evolution_recente_pct',
    'score_momentum',
    'est_quartier_emergent',
    'est_quartier_stable',
    'est_en_hausse'
]

colonnes_a_joindre = [col for col in colonnes_evolution if col in evolution_globale.columns]

dvf_enrichi = dvf_enrichi.merge(
    evolution_globale[colonnes_a_joindre],
    on='code_commune',
    how='left'
)

taux_match = dvf_enrichi['evolution_prix_m2_pct'].notna().mean() * 100
print(f"✅ {taux_match:.1f}% des transactions ont des données d'évolution")

# ═══════════════════════════════════════════════════════════
# 8. STATISTIQUES & VISUALISATION
# ═══════════════════════════════════════════════════════════

print("\n📊 STATISTIQUES ÉVOLUTION DES PRIX :\n")

print("Global (toutes communes) :")
print(f"  Évolution médiane : {evolution_globale['evolution_prix_m2_pct'].median():.1f}%")
print(f"  Croissance annuelle médiane : {evolution_globale['croissance_annuelle_pct'].median():.2f}%/an")
print(f"  Volatilité médiane : {evolution_globale['volatilite_pct'].median():.1f}%")

print("\n📈 TOP 5 communes - Plus forte hausse :")
top_hausse = evolution_globale.nlargest(5, 'evolution_prix_m2_pct')[
    ['code_commune', 'evolution_prix_m2_pct', 'croissance_annuelle_pct', 'prix_m2_debut', 'prix_m2_fin']
]
print(top_hausse.to_string(index=False))

print("\n📉 TOP 5 communes - Plus forte baisse :")
top_baisse = evolution_globale.nsmallest(5, 'evolution_prix_m2_pct')[
    ['code_commune', 'evolution_prix_m2_pct', 'croissance_annuelle_pct', 'prix_m2_debut', 'prix_m2_fin']
]
print(top_baisse.to_string(index=False))

print("\n🚀 Quartiers émergents (hausse forte + stable) :")
emergents = evolution_globale[evolution_globale['est_quartier_emergent'] == 1].nlargest(10, 'croissance_annuelle_pct')[
    ['code_commune', 'croissance_annuelle_pct', 'volatilite_pct', 'prix_m2_debut', 'prix_m2_fin']
]
print(emergents.to_string(index=False))

print("\n📊 Répartition par tendance :")
print(evolution_globale['tendance_categorie'].value_counts())

print("\n📊 Par département :")
if 'code_departement' in dvf_enrichi.columns:
    stats_dept = dvf_enrichi.groupby('code_departement').agg({
        'evolution_prix_m2_pct': 'median',
        'croissance_annuelle_pct': 'median',
        'volatilite_pct': 'median'
    }).round(2)
    stats_dept.columns = ['Evol_%', 'Croiss_an_%', 'Volat_%']
    print(stats_dept)

print("\n✅ COLONNES AJOUTÉES :")
print("   - evolution_prix_m2_euros : Variation absolue (€/m²)")
print("   - evolution_prix_m2_pct : Variation relative (%)")
print("   - croissance_annuelle_pct : TCAM (%/an)")
print("   - volatilite_pct : Écart-type variations")
print("   - tendance_lineaire_euros_an : Pente régression (€/an)")
print("   - tendance_categorie : Classification tendance")
print("   - evolution_recente_pct : Variation 2023-2025 (%)")
print("   - score_momentum : Score 0-100 (dynamique récente)")
print("   - est_quartier_emergent : 1 si hausse forte + stable")
print("   - est_quartier_stable : 1 si faible volatilité")
print("   - est_en_hausse : 1 si tendance haussière")

print("\n🎯 Utilité pour Clara :")
print("   ✅ Besoin 1 : Évolution prix 5 ans ✓")
print("   ✅ Besoin 2 : Détection quartiers émergents ✓")
print("   ✅ Besoin 3 : Potentiel valorisation ✓")
print("   ✅ Identification zones dynamiques")
print("   ✅ Analyse tendances historiques")
print("   ✅ Mesure risque (volatilité)")


📈 CALCUL ÉVOLUTION DES PRIX

📊 Calcul des prix médians par commune et année...

✅ 336 lignes (commune × année)
   Communes uniques : 56
   Période : 2020-2025
   Communes avec ≥3 années de données : 56

📊 Calcul évolution globale par commune...

✅ Évolution calculée pour 56 communes

📊 Calcul volatilité des prix...

✅ Volatilité calculée

📊 Détection des tendances (régression)...

✅ Tendances calculées

📊 Répartition tendances :
tendance_categorie
baisse          35
forte_baisse    14
stable           7
Name: count, dtype: int64

📊 Calcul évolution récente (2023-2025)...

✅ Évolution récente calculée pour 56 communes

🎯 Création des scores...

✅ Scores créés

🔗 Jointure avec DVF enrichi...



C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\2931770013.py:48: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  premiere_annee = evolution_prix.groupby('code_commune').apply(
C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\2931770013.py:52: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  derniere_annee = evolution_prix.groupby('code_commune').apply(
C:\Users\kombi\AppData\Local\Temp\ipykernel_12560\2931770013.py:14

✅ 100.0% des transactions ont des données d'évolution

📊 STATISTIQUES ÉVOLUTION DES PRIX :

Global (toutes communes) :
  Évolution médiane : -10.7%
  Croissance annuelle médiane : -2.23%/an
  Volatilité médiane : 4.4%

📈 TOP 5 communes - Plus forte hausse :
code_commune  evolution_prix_m2_pct  croissance_annuelle_pct  prix_m2_debut  prix_m2_fin
       92002                   4.24                     0.83        5400.00      5628.72
       75107                   1.94                     0.38       14874.17     15162.57
       92060                  -1.95                    -0.39        5802.47      5689.45
       75106                  -2.94                    -0.59       15750.00     15287.10
       92022                  -3.40                    -0.69        5862.07      5662.50

📉 TOP 5 communes - Plus forte baisse :
code_commune  evolution_prix_m2_pct  croissance_annuelle_pct  prix_m2_debut  prix_m2_fin
       92007                 -85.09                   -31.65       35500.00    

---
## 💾 ÉTAPE 9 : EXPORT FINAL

In [59]:
dvf_enrichi.columns

Index(['id_mutation', 'date_mutation', 'nature_mutation', 'code_departement',
       'code_commune', 'nom_commune', 'code_postal', 'latitude', 'longitude',
       'type_local', 'valeur_fonciere', 'surface_reelle_bati',
       'nombre_pieces_principales', 'prix_m2', 'annee', 'trimestre',
       'type_bien', 'population', 'revenu_median', 'taux_pauvrete',
       'nb_espaces_verts', 'superficie_espaces_verts_m2',
       'superficie_moyenne_espaces_verts_m2', 'diversite_espaces_verts',
       'nb_grands_parcs', 'score_espaces_verts', 'score_espaces_verts_norm',
       'est_quartier_vert', 'code_commune_temp', 'loyer_moyen_m2_mois',
       'loyer_mensuel_estime', 'loyer_annuel_estime_v2',
       'rendement_brut_pct_v2', 'profil_emprunteur', 'taux_interet_pct',
       'montant_emprunte', 'apport_initial', 'mensualite', 'cout_total_credit',
       'cout_total_acquisition', 'cashflow_mensuel', 'taux_couverture_pct',
       'est_cashflow_positif', 'est_effort_modere', 'est_taux_favorable',
    

In [60]:
print("💾 Sauvegarde du dataset final...\n")

# Créer dossier de sortie
output_dir = Path('../data_processed')
output_dir.mkdir(exist_ok=True)

# Sauvegarder en Parquet (efficace)
output_parquet = output_dir / 'dvf_population_75_92_final.parquet'
dvf_enrichi.to_parquet(output_parquet, index=False)
print(f"✅ Parquet : {output_parquet}")

# Sauvegarder en CSV (inspection)
output_csv = output_dir / 'dvf_population_75_92_final.csv'
dvf_enrichi.to_csv(output_csv, index=False, sep=';')
print(f"✅ CSV : {output_csv}")

print(f"\n📁 Fichiers générés :")
print(f"  - {len(dvf_enrichi):,} lignes")
print(f"  - {len(dvf_enrichi.columns)} colonnes")
print(f"  - Taille Parquet : {output_parquet.stat().st_size / 1024 / 1024:.1f} Mo")

💾 Sauvegarde du dataset final...

✅ Parquet : ..\data_processed\dvf_population_75_92_final.parquet
✅ CSV : ..\data_processed\dvf_population_75_92_final.csv

📁 Fichiers générés :
  - 279,554 lignes
  - 56 colonnes
  - Taille Parquet : 28.2 Mo


---
## ✅ RÉSUMÉ FINAL

In [63]:
dvf_enrichi.to_excel("C:/Users/kombi/OneDrive/Bureau/p/Projet_Data_Science/projet data science/outputs/dvf_population_75_92_final.xlsx", index=False)